In [1]:
import os
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

2025-05-05 01:10:21.890947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746407422.151814      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746407422.232029      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def load_and_split_image_data(
    data_dir: str,
    test_size: float = 0.1,
    val_size: float = 0.2,
    batch_size: int = 32,
    img_size: tuple = (120, 160),
    random_state: int = 42
):
    """
    Load images from class-named subfolders, exclude 'Sample...' folder, preprocess, and split.

    Returns tf.data.Dataset objects for image-only data.
    """

    # find class directories
    class_dirs = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))
                  and not d.startswith("Sample")]
    label_map = {name: idx for idx, name in enumerate(sorted(class_dirs))}

    paths, labels = [], []
    for cls in class_dirs:
        folder = os.path.join(data_dir, cls)
        imgs = glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.png"))
        paths.extend(imgs)
        labels.extend([label_map[cls]] * len(imgs))
    paths = np.array(paths)
    labels = np.array(labels)

    X_temp, X_test, y_temp, y_test = train_test_split(
        paths, labels, test_size=test_size, stratify=labels, random_state=random_state
    )
    val_frac = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_frac, stratify=y_temp, random_state=random_state
    )

    def process_image(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    def make_ds(paths, labels, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))
        ds = ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(buffer_size=len(paths), seed=random_state)
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_ds(X_train, y_train, shuffle=True)
    val_ds   = make_ds(X_val,   y_val,   shuffle=False)
    test_ds  = make_ds(X_test,  y_test,  shuffle=False)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = load_and_split_image_data("../dataset/Thermal Camera Images")

2025-05-05 01:10:40.474537: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
class MCDropout(tf.keras.layers.Dropout):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference
        return super().call(inputs, training=True)
    

class MCSpatialDropout2D(tf.keras.layers.SpatialDropout2D):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference 
        return super().call(inputs, training=True)


In [5]:
def build_image_model(
    img_shape=(120,160,3),
    conv_filters=(32, 64 , 128),
    layer_dropout = 0.5,
    dense_units=64,
    output_units=4,
    lr=1e-4
):
    """
    Image-only CNN classifier.
    """
    inp = tf.keras.Input(shape=img_shape, name="image_input")
    x = tf.keras.layers.Conv2D(conv_filters[0], 3, activation="relu", padding="same", use_bias=False)(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[1], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(conv_filters[2], 3, activation="relu", padding="same",use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = MCSpatialDropout2D(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(dense_units, activation="relu")(x)
    x = MCDropout(layer_dropout)(x)
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(x)

    model = tf.keras.Model(inp, out, name="image_only")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [6]:
def predict_mc(model, dataset, T=50):
    all_preds = []
    all_labels = []
    for sens_batch, lbl_batch in dataset:
        preds_t = [model(sens_batch).numpy() for _ in range(T)]
        preds_t = np.stack(preds_t, axis=0)  # (T, batch, classes)
        mean_preds = preds_t.mean(axis=0)    # (batch, classes)
        all_preds.append(mean_preds)
        all_labels.append(lbl_batch.numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    y_pred = np.argmax(all_preds, axis=1)
    return np.mean(y_pred == all_labels)

In [7]:
model = build_image_model()
ckpt_path = "best_image_dropout.keras"
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        verbose=0
    )
model.fit(train_ds, validation_data=val_ds, epochs=100 , callbacks=[checkpoint_cb])
best_model = tf.keras.models.load_model(
        ckpt_path,
        custom_objects={"MCSpatialDropout2D": MCSpatialDropout2D, "MCDropout": MCDropout}
    )
acc = predict_mc(best_model, test_ds, T=50)
print("testing accuracy is ",acc)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.5044 - loss: 1.1376 - val_accuracy: 0.2438 - val_loss: 1.6429
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.6810 - loss: 0.7822 - val_accuracy: 0.2859 - val_loss: 2.3929
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.7031 - loss: 0.6939 - val_accuracy: 0.3492 - val_loss: 2.1327
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.7477 - loss: 0.6227 - val_accuracy: 0.5570 - val_loss: 1.2871
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.7584 - loss: 0.6187 - val_accuracy: 0.6805 - val_loss: 0.8169
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.7705 - loss: 0.5750 - val_accuracy: 0.7945 - val_loss: 0.5263
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.7939 - loss: 0.5274 - val_accuracy: 0.7961 - val_loss: 0.5082
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 181s 1s/step - accuracy: 0.8020 - loss: 0.5062 - 